# Face - Feature Detection Model Training

This notebook provides a clean, organized pipeline for training models for Face detection/feature estimation.
It follows the same modular structure as the LE (Left Eye) experiment.

## Experiment Overview
- **Dataset**: Face images with feature labels
- **Task**: Regress two continuous values (feature coordinates)
- **Approaches**: Baseline model, Transfer Learning, From Scratch
- **Note**: Same architecture and training strategy as LE, applied to Face data

## Section 1: Setup & Imports

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Import custom modules
from config import get_config
from data_loader import load_images, CustomDataGenerator, get_augmentation_generator
from model_architecture import create_and_compile_model
from training_utils import train_model, plot_training_results, save_history_to_json, get_callbacks
from system_monitoring import print_system_info

print("All imports successful!")

## Section 2: System Information & Configuration

In [ ]:
# Print system information
print_system_info()

# Get experiment configuration for Face
experiment = 'Face'
config = get_config(experiment)

print(f"\nExperiment: {config['experiment_info']['name']}")
print(f"Image Size: {config['image_size']}")
print(f"Batch Size: {config['batch_size']}")
print(f"Epochs: {config['epochs']}")
print(f"Learning Rate: {config['learning_rate']}")

## Section 3: Load Data

In [ ]:
# Get experiment info
exp_info = config['experiment_info']

print("Loading baseline training data...")
train_samples, train_labels = load_images(
    exp_info['baseline'],
    image_size=config['image_size']
)

print("\nLoading test data...")
test_samples, test_labels = load_images(
    exp_info['test'],
    image_size=config['image_size']
)

print(f"\nTrain samples shape: {train_samples.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_samples.shape}")
print(f"Test labels shape: {test_labels.shape}")

## Section 4: Create Data Generators

In [ ]:
# Create augmentation generator
datagen = get_augmentation_generator()

# Create custom data generators
train_generator = CustomDataGenerator(
    train_samples, train_labels,
    batch_size=config['batch_size'],
    shuffle=True,
    datagen=datagen
)

test_generator = CustomDataGenerator(
    test_samples, test_labels,
    batch_size=config['batch_size'],
    shuffle=False,
    datagen=None
)

print(f"Training batches per epoch: {len(train_generator)}")
print(f"Test batches per epoch: {len(test_generator)}")

## Section 5: Train Baseline Model

In [ ]:
# Create model
model = create_and_compile_model(
    input_shape=(config['image_size'], config['image_size'], 3),
    dense_units=96,
    dropout_rate=config['dropout_rate'],
    learning_rate=config['learning_rate']
)

print("Model created and compiled!")
model.summary()

In [ ]:
# Get callbacks
callbacks = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

# Train model
results = train_model(
    model,
    train_generator,
    test_generator,
    epochs=config['epochs'],
    callbacks=callbacks,
    model_name="participant_baseline"
)

history = results['history']
training_time = results['time']

print(f"\nTraining completed in {training_time['hours']:.2f} hours")

## Section 6: Visualize Training Results

In [ ]:
# Plot training results
plot_training_results(history, model_name="(participant_baseline)")

In [ ]:
# Load Ola dataset (example with 500 samples)
dataset_key = '500'  # Options: '100', '200', '300', '400', '500'

print(f"Loading Ola-augmented data ({dataset_key} samples)...")
ola_train_samples, ola_train_labels = load_images(
    exp_info['training'][dataset_key],
    image_size=config['image_size']
)

print(f"Ola train samples shape: {ola_train_samples.shape}")
print(f"Ola train labels shape: {ola_train_labels.shape}")

## Section 7: Save Model & Results

In [ ]:
# Save model
model_name = 'Face_Baseline'
model.save(f'{model_name}.keras')
print(f"Model saved as {model_name}.keras")

# Save training history
save_history_to_json(history, f'{model_name}.json')

## Section 8: Transfer Learning with Different Dataset Sizes

Fine-tune the baseline model with different amounts of Ola-augmented data.

In [ ]:
# Save model
model_name = 'participant_baseline'
model.save(f'{model_name}.keras')
print(f"Model saved as {model_name}.keras")

# Save training history
save_history_to_json(history, f'{model_name}.json')

In [ ]:
# Create generators for transfer learning
ola_train_gen = CustomDataGenerator(
    ola_train_samples, ola_train_labels,
    batch_size=config['batch_size'],
    shuffle=True
)

ola_test_gen = CustomDataGenerator(
    test_samples, test_labels,
    batch_size=config['batch_size'],
    shuffle=False
)

# Load baseline model for fine-tuning
from tensorflow.keras.models import load_model
model_ft = load_model(f'{model_name}.keras')

# Fine-tune
callbacks_ft = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

results_ft = train_model(
    model_ft,
    ola_train_gen,
    ola_test_gen,
    epochs=config['epochs'],
    callbacks=callbacks_ft,
    model_name=f"participant_transfer_{dataset_key}"
)

history_ft = results_ft['history']
print(f"\nTransfer learning completed")

In [ ]:
# Plot transfer learning results
plot_training_results(history_ft, model_name=f"(Face Transfer - {dataset_key} Ola)")

## Section 9: From-Scratch Training with Ola Data

In [ ]:
# Create new model for from-scratch training
model_scratch = create_and_compile_model(
    input_shape=(config['image_size'], config['image_size'], 3),
    dense_units=96,
    dropout_rate=config['dropout_rate'],
    learning_rate=config['learning_rate']
)

callbacks_scratch = get_callbacks(
    patience_early_stopping=config['patience_early_stopping'],
    patience_reduce_lr=config['patience_reduce_lr'],
    factor_reduce_lr=config['factor_reduce_lr'],
    min_learning_rate=config['min_learning_rate']
)

results_scratch = train_model(
    model_scratch,
    ola_train_gen,
    ola_test_gen,
    epochs=config['epochs'],
    callbacks=callbacks_scratch,
    model_name=f"participant_scratch_{dataset_key}"
)

history_scratch = results_scratch['history']
print(f"\nFrom-scratch training completed")

In [ ]:
# Compare all three approaches
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_baseline = range(1, len(history.history['val_loss']) + 1)
epochs_ft = range(1, len(history_ft.history['val_loss']) + 1)
epochs_scratch = range(1, len(history_scratch.history['val_loss']) + 1)

# Validation Loss
axes[0].plot(epochs_baseline, history.history['val_loss'], 'b-o', label='Baseline', linewidth=2, markersize=3)
axes[0].plot(epochs_ft, history_ft.history['val_loss'], 'g-s', label=f'Transfer (Ola-{dataset_key})', linewidth=2, markersize=3)
axes[0].plot(epochs_scratch, history_scratch.history['val_loss'], 'r-^', label=f'From-Scratch (Ola-{dataset_key})', linewidth=2, markersize=3)
axes[0].set_title('Validation Loss Comparison', fontsize=14)
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True)

# Validation MAE
axes[1].plot(epochs_baseline, history.history['val_mae'], 'b-o', label='Baseline', linewidth=2, markersize=3)
axes[1].plot(epochs_ft, history_ft.history['val_mae'], 'g-s', label=f'Transfer (Ola-{dataset_key})', linewidth=2, markersize=3)
axes[1].plot(epochs_scratch, history_scratch.history['val_mae'], 'r-^', label=f'From-Scratch (Ola-{dataset_key})', linewidth=2, markersize=3)
axes[1].set_title('Validation MAE Comparison', fontsize=14)
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Section 10: Conclusions & Notes

- Same architecture and training strategy as the LE (Left Eye) experiment
- **Baseline Model**: Trained on data without Ola augmentation
- **Transfer Learning**: Fine-tunes baseline with Ola-augmented data
- **From-Scratch**: Trains new model directly on Ola-augmented data

Compare the results to determine which approach works best for Face region feature detection.